<h2>Step 1: Importing required stuff:</h2>

In [9]:
pip install beautifulsoup4

     |████████████████████████████████| 112kB 14.5MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

<h2>Step 2: Scraping the Wikipedia Page:</h2>

In [11]:
#To download the Wikipedia page:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = urllib.request.urlopen(url)
article = req.read().decode()

In [12]:
#Fetching the table:
soup = BeautifulSoup(article, 'html.parser')
table = soup.find('table', class_='sortable')

In [13]:
#To parse the table into a dataframe:
#headings:
ths = table.find_all('th')
headings = [th.text.strip() for th in ths]

#create dataframe:
neighbourhoods = pd.DataFrame(columns=headings)
neighbourhoods

#scrape rows into dataframe:
for tr in table.find_all('tr'):
    tds = tr.find_all('td')
    if not tds:
        continue                            #skips first row with headings
    pc, b, n = [td.text.strip() for td in tds]
    if b!="Not assigned":
        neighbourhoods = neighbourhoods.append({'Postcode': pc, 'Borough': b, 'Neighbourhood': n}, ignore_index=True)

neighbourhoods.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


<h2>Step 3: Cleaning up the data:</h2>

In [14]:
#Combine neighbourhoods for the same postcode:
neighbourhoods['Neighbourhood'] = neighbourhoods.groupby('Postcode')['Neighbourhood'].transform(lambda x: "%s" % ', '.join(x)).values

#Drop all duplicates:
neighbourhoods = neighbourhoods.drop_duplicates().reset_index(drop=True)
neighbourhoods.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [15]:
#To replace Neighbourhoods with 'Not assigned' to their Burough name:
neighbourhoods['Neighbourhood'].replace("Not assigned", neighbourhoods['Borough'], inplace=True)
neighbourhoods.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [17]:
neighbourhoods.shape

(103, 3)